<a href="https://colab.research.google.com/github/kusawokoshitemori/ViT_STL10/blob/main/ViT_STL10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 734.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import os

# 1. データ準備
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT用にリサイズ
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# データセットのダウンロードとロード
# データセットがダウンロードされるディレクトリを作成
data_dir = './data'
train_dataset = datasets.STL10(root=data_dir, split='train', download=True, transform=transform)
test_dataset = datasets.STL10(root=data_dir, split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import os

# 2. ViTモデルの定義
# Patch Embedding: 画像を小さなパッチに分割し、線形に埋め込む
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=16, embed_dim=768):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        # x: (batch_size, in_channels, img_h, img_w)
        x = self.proj(x) # (batch_size, embed_dim, num_patches_h, num_patches_w)
        x = x.flatten(2) # (batch_size, embed_dim, num_patches) - 高さx幅の次元を平坦化
        x = x.transpose(1, 2) # (batch_size, num_patches, embed_dim) - Transformerの入力形式に合わせる
        return x

# Multi-Head Self-Attention: 入力シーケンスの異なる部分に同時に注意を払う
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads # 各ヘッドの次元
        self.scale = self.head_dim ** -0.5 # スケーリングファクター

        self.wq = nn.Linear(embed_dim, embed_dim) # Queryの線形変換
        self.wk = nn.Linear(embed_dim, embed_dim) # Keyの線形変換
        self.wv = nn.Linear(embed_dim, embed_dim) # Valueの線形変換
        self.out = nn.Linear(embed_dim, embed_dim) # 出力の線形変換

    def forward(self, x):
        batch_size, num_tokens, embed_dim = x.shape

        # Query, Key, Valueの計算とヘッドごとの分割
        # 各テンソルは (batch_size, num_heads, num_tokens, head_dim) の形になる
        q = self.wq(x).view(batch_size, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.wk(x).view(batch_size, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.wv(x).view(batch_size, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)

        # Attentionスコアの計算: QとKの行列積
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        # Attention確率の計算: softmaxを適用
        attention_probs = torch.softmax(attention_scores, dim=-1)

        # 加重平均値の計算: Attention確率とVの行列積
        output = torch.matmul(attention_probs, v)
        # 各ヘッドの出力を結合し、元の埋め込み次元に戻す
        output = output.transpose(1, 2).contiguous().view(batch_size, num_tokens, embed_dim)
        # 最終的な線形変換
        output = self.out(output)
        return output

# Transformer Encoder Block: Multi-Head Self-Attention と Feed-Forward Network を含む
class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim) # Attention前のLayerNorm
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads) # Multi-Head Self-Attention
        self.norm2 = nn.LayerNorm(embed_dim) # MLP前のLayerNorm
        self.mlp = nn.Sequential( # Feed-Forward Network
            nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
            nn.GELU(), # 活性化関数
            nn.Dropout(dropout),
            nn.Linear(int(embed_dim * mlp_ratio), embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # 残差接続とLayerNorm、Attention
        x = x + self.attn(self.norm1(x))
        # 残差接続とLayerNorm、MLP
        x = x + self.mlp(self.norm2(x))
        return x

# Vision Transformer (ViT) モデル全体
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, num_classes=10,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., dropout=0.1):
        super().__init__()
        # 画像のパッチ数
        num_patches = (img_size // patch_size) * (img_size // patch_size)

        self.patch_embed = PatchEmbedding(in_channels, patch_size, embed_dim) # パッチ埋め込み層

        # クラス分類トークン (学習可能なパラメータ)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        # 位置エンコーディング (学習可能なパラメータ) - クラストークンの分も+1
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(dropout) # 位置エンコーディングに適用するドロップアウト

        # Transformerエンコーダーブロックのリスト
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(depth)
        ])

        self.norm = nn.LayerNorm(embed_dim) # 最終LayerNorm
        self.head = nn.Linear(embed_dim, num_classes) # 分類ヘッド

    def forward(self, x):
        x = self.patch_embed(x) # パッチ埋め込み

        # クラストークンをバッチサイズに合わせて拡張し、パッチ埋め込みと結合
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1) # クラストークンをシーケンスの先頭に追加

        x = x + self.pos_embed # 位置エンコーディングを加算
        x = self.pos_drop(x) # ドロップアウト適用

        # Transformerエンコーダーブロックを順に適用
        for block in self.blocks:
            x = block(x)

        x = self.norm(x) # 最終LayerNorm
        return self.head(x[:, 0]) # クラストークンに対応する出力を使用して分類

# 3. モデルのインスタンス化、損失関数、最適化
# GPUが利用可能かチェック
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")

# ViTモデルをインスタンス化（FashionMNISTは10クラス）
# 例として、モデルのdepthを小さくして（例: 4）、学習時間を短縮できます。
model = VisionTransformer(num_classes=10, depth=4, embed_dim=256, num_heads=8).to(device)
print(f"モデルのパラメータ数: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

criterion = nn.CrossEntropyLoss() # 損失関数
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # オプティマイザ

# 4. 学習ループ
num_epochs = 5 # 例としてエポック数を5に設定

print("\n--- 学習開始 ---")
for epoch in range(num_epochs):
    model.train() # モデルを訓練モードに設定
    running_loss = 0.0
    start_time = time.time() # エポック開始時刻を記録

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device) # データをデバイスに転送

        optimizer.zero_grad() # 勾配をゼロにリセット
        outputs = model(images) # 順伝播
        loss = criterion(outputs, labels) # 損失計算
        loss.backward() # 逆伝播（勾配計算）
        optimizer.step() # パラメータ更新

        running_loss += loss.item() # 損失を加算

        # 進捗表示 (例: 100バッチごとに表示)
        if (i + 1) % 100 == 0:
            print(f"  Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/(i+1):.4f}, Time per batch: {(time.time() - start_time)/(i+1):.4f}s")

    end_time = time.time() # エポック終了時刻を記録
    print(f"Epoch [{epoch+1}/{num_epochs}], 平均損失: {running_loss/len(train_loader):.4f}, エポック時間: {end_time - start_time:.2f}s")

    # 5. 評価ループ
    model.eval() # モデルを評価モードに設定
    correct = 0
    total = 0
    with torch.no_grad(): # 勾配計算を無効化
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) # 最も確率の高いクラスを予測
            total += labels.size(0) # 合計サンプル数
            correct += (predicted == labels).sum().item() # 正解数をカウント

    print(f"テスト画像での精度: {(100 * correct / total):.2f}%")

print("\n--- 学習終了 ---")


100%|██████████| 2.64G/2.64G [02:42<00:00, 16.2MB/s]


使用デバイス: cuda
モデルのパラメータ数: 3409674

--- 学習開始 ---
Epoch [1/5], 平均損失: 2.0277, エポック時間: 21.62s
テスト画像での精度: 28.85%
Epoch [2/5], 平均損失: 1.8085, エポック時間: 19.15s
テスト画像での精度: 35.04%
Epoch [3/5], 平均損失: 1.6670, エポック時間: 20.11s
テスト画像での精度: 37.15%
Epoch [4/5], 平均損失: 1.5886, エポック時間: 19.67s
テスト画像での精度: 38.62%
Epoch [5/5], 平均損失: 1.5311, エポック時間: 20.09s
テスト画像での精度: 37.06%

--- 学習終了 ---
